In [71]:
import torch
import torch.utils.data as data

In [62]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
from ignite.contrib.handlers.wandb_logger import *

In [72]:
data_rand = torch.rand((4,5))

In [251]:
data_loader = data.DataLoader(data_rand, batch_size=2)
data_eval_loader = data.DataLoader(data_rand, batch_size=4)

In [232]:
len(data_loader)

2

In [252]:
len(data_eval_loader)

1

In [285]:
# aa = 0
def process_func(trainer, batch):
#     print('do_training')
    a = 0
    
def pro_func_eval(trainer, batch):
#     aa +=1
    print("doing eval")

trainer = Engine(process_func)
evaler = Engine(pro_func_eval)

In [286]:
# @trainer.on(Events.ITERATION_COMPLETED)
def every_iter():
    print('end_of_iteration')

In [287]:
@trainer.on(Events.EPOCH_COMPLETED)
def run_eval():
    evaler.run(data_eval_loader)

In [288]:
@trainer.on(Events.COMPLETED)
def test_eval():
    print('do_test_eval')

In [289]:
def score_func(engine):
    lost = engine.state.iteration
    print('score_func',-lost)
    return -lost

handler = EarlyStopping(patience=3, score_function=score_func, trainer=trainer)
evaler.add_event_handler(Events.COMPLETED, handler)

In [290]:
trainer.run(data_loader, max_epochs=20)

2022-06-29 16:48:39,831 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


doing eval
score_func -1
doing eval
score_func -1
doing eval
score_func -1
doing eval
score_func -1
do_test_eval


State:
	iteration: 8
	epoch: 4
	epoch_length: 2
	max_epochs: 20
	output: <class 'NoneType'>
	batch: <class 'torch.Tensor'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>